Let's solve a rubik's cube.

In [14]:
import numpy as np

class Cube:
    def __init__(self):
        self.corners = np.array([np.eye(3)[0] for _ in range(8)])
        self.edges = np.array([np.eye(2)[0] for _ in range(12)])

    def data(self):
        return np.concatenate([np.concatenate(self.corners,0),np.concatenate(self.edges,0)],0)

    def rotate(self):
        pass

In [15]:
c = Cube()

In [16]:
c.rotate()